In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

In [4]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [5]:
test = pd.read_csv(TEST)
test_ids = test['id']

In [6]:
# Carga de datos ya procesados
X=np.load(PREPROCESSED_DATA+'binScale.npz')
train = X['arr_0']
label = X['arr_1']
test = X['arr_2']
X.close()

In [7]:
train.shape

(9125, 52)

In [8]:
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.1)
#mlp=MLPClassifier(hidden_layer_sizes=100, early_stopping=True, random_state=25, max_iter=1000)
xgb=XGBClassifier(n_estimators=200, max_depth=3, n_jobs=4, eval_metric='mlogloss')
#hist=HistGradientBoostingClassifier(max_iter=75,max_leaf_nodes=31,learning_rate=0.1)

lr=LogisticRegression(max_iter=1000)

estimators=[('gb',gb),('xgb',xgb)]
model=StackingClassifier(estimators=estimators,final_estimator=lr,n_jobs=4,verbose=100)

In [9]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

[0.91616438 0.92547945 0.91945205 0.93205479 0.91945205]
0.9225205479452054


## Generar fichero de Kaggle

In [10]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try33.csv", index=False)